In [1]:
#Import Data Set and Python Libraries
import math, time, random, datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-talk')
import seaborn as sns
sns.set(color_codes=True)
sns.set_style('white')


from google.colab import drive
drive.mount('/content/drive')

path = "drive/My Drive/Colab Notebooks/data/energy/"

NYC_data = pd.read_excel(path+"NYC_2020_Data_Updated.xlsx")

Mounted at /content/drive
